In [1]:
# import required libraries
import os
import numpy as np
from PIL import Image as im

# Ruta a los datos y las clases
dataPath = '/home/saksevul/T-MCC/OpenData/'
# ruta al tipo de jets a considerar
jetsType = 'Jets'
# jetsType = 'Random'
# directorio donde se ubican los archivos txt
txtFiles = jetsType+'/txtFiles/'

# Lista de los AODs dentro de path
aod = ['Jet', 'MultiJet']

# Ordensmos alfabéticamente
aod.sort()

# Lista de archivos con datos
eFile = []
hFile = []

# tamaño de las imagenes
eN = 34
eM = 72
hN = 34
hM = 72
lN = 34
lM = 72

# Encuentra los archivos de datos contenidos en los directorios
for i in os.listdir(dataPath+aod[0]+'_20000/'+txtFiles):
    if i[:5]=='ECal'+'-' and i[-4:]=='.txt':
        eFile.append(i[5:9])
    if i[:5]=='HCal'+'-' and i[-4:]=='.txt':
        hFile.append(i[5:9])

# reordenar los valores
eFile.sort()
hFile.sort()

print("eFile: ", len(eFile), eFile)
print("hFile: ", len(hFile), hFile)

eFile:  50 ['0006', '0007', '0008', '0012', '0014', '0017', '0020', '0021', '0024', '0034', '0035', '0039', '0040', '0042', '0044', '0045', '0060', '0064', '0066', '0067', '0070', '0072', '0081', '0083', '0084', '0085', '0090', '0093', '0094', '0101', '0107', '0109', '0112', '0116', '0117', '0120', '0122', '0123', '0124', '0127', '0129', '0135', '0136', '0139', '0145', '0147', '0152', '0154', '0157', '0158']
hFile:  50 ['0006', '0007', '0008', '0012', '0014', '0017', '0020', '0021', '0024', '0034', '0035', '0039', '0040', '0042', '0044', '0045', '0060', '0064', '0066', '0067', '0070', '0072', '0081', '0083', '0084', '0085', '0090', '0093', '0094', '0101', '0107', '0109', '0112', '0116', '0117', '0120', '0122', '0123', '0124', '0127', '0129', '0135', '0136', '0139', '0145', '0147', '0152', '0154', '0157', '0158']


In [6]:
# Extraer los datos necesarios

# semilla para los números aleatorios
np.random.seed(100)

# Porción de datos de entrenamiento
p = 0.5

# Definimos el conjunto de datos para el ECal, así como sus clases
for i in range(len(aod)):
    print(aod[i])
    for j in range(len(eFile)):
        if i==0 and j==0:
            eData = np.loadtxt(dataPath+aod[i]+'_20000/'+txtFiles+'ECal'+'-'+eFile[j]+'.txt')
            hData = np.loadtxt(dataPath+aod[i]+'_20000/'+txtFiles+'HCal'+'-'+hFile[j]+'.txt')
            jet = np.loadtxt(dataPath+aod[i]+'_20000/'+txtFiles+'Jet'+'-'+hFile[j]+'.txt', usecols=(0,1,2,3,4,5,6,7,8,9,10,11))
            rows = eData.shape[i]
            classes = np.ones(rows)*i
            # Indices del conjunto de entrenamiento y prueba del primer conjunto de AODs
            trainInd = np.random.choice(np.array(range(len(eFile)*rows)),int(p*len(eFile)*rows),replace=False)
            testInd = np.setdiff1d(np.array(range(len(eFile)*rows)),trainInd)
        else:
            eData = eval('np.concatenate((eData,np.loadtxt('+"'"+dataPath+aod[i]+'_20000/'+txtFiles+'ECal'+'-'+eFile[j]+'.txt'+"'"+')), axis=0)')
            hData = eval('np.concatenate((hData,np.loadtxt('+"'"+dataPath+aod[i]+'_20000/'+txtFiles+'HCal'+'-'+hFile[j]+'.txt'+"'"+')), axis=0)')
            jet = eval('np.concatenate((jet,np.loadtxt('+"'"+dataPath+aod[i]+'_20000/'+txtFiles+'Jet'+'-'+hFile[j]+'.txt'+"', usecols=(0,1,2,3,4,5,6,7,8,9,10,11)"+')), axis=0)')
            classes = eval('np.concatenate((classes,np.ones(rows)*i), axis=0)')
    if i>0:
        trainInd = np.concatenate((trainInd, i*len(eFile)*rows+trainInd[:int(p*len(eFile)*rows)]), axis=0)
        testInd = np.concatenate((testInd, i*len(eFile)*rows+testInd[:int(((10-10*p)/10)*len(eFile)*rows)]), axis=0)


# Mexclar una última vez los índices
trainInd = np.random.choice(trainInd,trainInd.shape[0],replace=False)
testInd = np.random.choice(testInd,testInd.shape[0],replace=False)

print("eData ", eData.shape, type(eData))
#print(eData[0][:12])

print("hData ", hData.shape, type(hData))
#print(hData[0][:12])
 
print("jet ", jet.shape, type(jet))
print("jet ", jet[trainInd[1024]])

print("trainInd: ", trainInd[:12])
#trainInd:  [1859 1756 1375 1915 1353   54  247  960  985  302   78  411]

print("testInd: ", testInd[:12])
#testInd:  [1776  478 1821 1695 1168  138   38 1049 1170  524  802  148]


# Normalizar los datos y convertirlos en formato uint8 para convertirlos a imágenes PNG
lData = np.log(eData+hData+1)

# reescalamos todos los valores para ser re escritos en imágenes.
for i in range(eData.shape[0]):
    eMax = np.max(eData[i])
    hMax = np.max(hData[i])
    lMax = np.max(lData[i])
    eData[i] = 255*eData[i]/eMax
    hData[i] = 255*hData[i]/hMax
    lData[i] = 255*lData[i]/lMax

# cambiar el formato de los datos
eData = eData.astype(np.int8)
hData = hData.astype(np.int8)
lData = lData.astype(np.int8)


Jet
MultiJet
eData  (6400, 2448) <class 'numpy.ndarray'>
hData  (6400, 2448) <class 'numpy.ndarray'>
jet  (6400, 12) <class 'numpy.ndarray'>
jet  [-1.198 -0.595  1.706  0.888  2.335 -0.647  1.083  0.479 -0.675 -0.372
  2.769  0.584]
trainInd:  [2282 5913 4124 1029 5136 4321 4667 4989 3703 5935 3152 5427]
testInd:  [6386 5092 3948 2493 5744 3081 1357 4668 3619  430 3218 2978]


## Generar las imágenes

In [3]:
# Crear las imágenes una a una
# Rescale Factor
rF = 12

# numero de imagenes a generar
nImg = 2**10

# contador para las imagenes
crs = '0000'

# generar un archivo el cual contiene la información de los jets que se generarán en la imágenes.
with open("../calImages/"+jetsType+"/Jet.txt", "w") as JetF:
    # ciclo sobre todas las imagenes a generar
    for i in range(nImg):
        # imprimir la información de la imagen a generar
        if (i%(nImg/16)==0):
            print("  ",crs[:-len(str(i+1))]+str(i+1),".jpg images")
        # Reescalar los arreglos a partir de los cuales se generarán las imágines))
#         eD = np.ones((rF*eN,rF*eM,3), dtype=np.uint8)
#         hD = np.ones((rF*hN,rF*hM,3), dtype=np.uint8)
        lD = np.ones((rF*lN,rF*lM,3), dtype=np.uint8)
        eDat = eData[trainInd[i]].reshape(eN,eM)
        hDat = hData[trainInd[i]].reshape(hN,hM)
        lDat = lData[trainInd[i]].reshape(lN,lM)
        for j in range(rF*eN):
            for k in range(rF*eM):
#                 for l in range(3):
#                     eD[j][k][l] = eD[j][k][l]*eDat[int(j/rF)][int(k/rF)]
#                     hD[j][k][l] = hD[j][k][l]*hDat[int(j/rF)][int(k/rF)]
                lD[j][k][0] = lD[j][k][0]*eDat[int(j/rF)][int(k/rF)]
                lD[j][k][1] = lD[j][k][1]*hDat[int(j/rF)][int(k/rF)]
                lD[j][k][2] = lD[j][k][2]*lDat[int(j/rF)][int(k/rF)]
        #eD = np.kron(eData[trainInd[i]].reshape(eN,eM), np.ones((rF,rF), dtype=np.int8))
        # Generar las imágenes
#         im.fromarray(eD, mode='RGB').save('../calImages/'+jetsType+'/ECal_'+crs[:-len(str(i+1))]+str(i+1)+'.jpg', quality=100, subsampling=0)
#         im.fromarray(hD, mode='RGB').save('../calImages/'+jetsType+'/HCal_'+crs[:-len(str(i+1))]+str(i+1)+'.jpg', quality=100, subsampling=0)
        im.fromarray(lD, mode='RGB').save('../calImages/'+jetsType+'/Cals_'+crs[:-len(str(i+1))]+str(i+1)+'.jpg', quality=100, subsampling=0)
        if (i >= nImg-24):
#             im.fromarray(eD, mode='RGB').save('../jet-detection/img_pruebas/ECal_'+crs[:-len(str(i+1))]+str(i+1)+'.jpeg', format='JPEG', quality=100, subsampling=0)
#             im.fromarray(hD, mode='RGB').save('../jet-detection/img_pruebas/HCal_'+crs[:-len(str(i+1))]+str(i+1)+'.jpeg', format='JPEG', quality=100, subsampling=0)
            im.fromarray(lD, mode='RGB').save('../jet-detection/img_pruebas/Cals_'+crs[:-len(str(i+1))]+str(i+1)+'.jpeg', format='JPEG', quality=100, subsampling=0)
        # escribir en un archivo las coordenadas de los jets
        JetF.write("".join(str(jet[trainInd[i]])) + "\n")

# cerrar la imagen
JetF.close()

print("  Done! :D")

   0001 .jpg images
   0065 .jpg images
   0129 .jpg images
   0193 .jpg images
   0257 .jpg images
   0321 .jpg images
   0385 .jpg images
   0449 .jpg images
   0513 .jpg images
   0577 .jpg images
   0641 .jpg images
   0705 .jpg images
   0769 .jpg images
   0833 .jpg images
   0897 .jpg images
   0961 .jpg images
  Done! :D


## Generar los archivos .xml

In [7]:
# información .xml

# radio del jet
rJ = 0.5

# Lista de los AODs dentro de path
# cal = ['Cals', 'ECal', 'HCal']
cal = ['Cals']

# ciclo sobre todas las imagenes a generar
for i in range(1000,nImg):
    # imprimir la información de la imagen a generar
    if (i%(nImg/16)==0):
        print("  ",crs[:-len(str(i+1))]+str(i+1),".xml files")
    nJet = 0
    for k in cal:
        with open("../calImages/"+jetsType+"/"+k+"_"+crs[:-len(str(i+1))]+str(i+1)+".xml", "w") as xmlF:
            # preludio
            xmlF.write("".join("<annotation>")+"\n")
            xmlF.write("".join("  <folder>calImages</folder>")+"\n")
            xmlF.write("".join("  <filename>"+k+"_"+crs[:-len(str(i+1))]+str(i+1)+".jpg</filename>")+"\n")
            xmlF.write("".join("  <path>/home/saksevul/T-MCC/calImages/"+jetsType+"/"+k+"_"+crs[:-len(str(i+1))]+str(i+1)+".jpg</path>")+"\n")
            xmlF.write("".join("  <source>")+"\n")
            xmlF.write("".join("    <database>Unknown</database>")+"\n")
            xmlF.write("".join("  </source>")+"\n")
            xmlF.write("".join("  <size>")+"\n")
            xmlF.write("".join("    <width>"+str(rF*hM)+"</width>")+"\n")
            xmlF.write("".join("    <height>"+str(rF*hN)+"</height>")+"\n")
            xmlF.write("".join("    <depth>3</depth>")+"\n")
            xmlF.write("".join("  </size>")+"\n")
            xmlF.write("".join("  <segmented>0</segmented>")+"\n")
            # información de los jets
            for j in range(int(jet.shape[1]/2)):
                # evaluar repeticiones
                if (jet[trainInd[i]][2*j]>9 and jet[trainInd[i]][2*j+1]>9):
                    continue
                # bandera
                trunc = 0
                # transformación de coordenadas espaciales a pixeles
                x_min = int( rF*(jet[trainInd[i]][2*j+0]+np.pi-rJ)*hM/(2*np.pi) )
                x_max = int( rF*(jet[trainInd[i]][2*j+0]+np.pi+rJ)*hM/(2*np.pi) )
                y_min = int( rF*(1.479-jet[trainInd[i]][2*j+1]-rJ)*hN/(2*1.479) )
                y_max = int( rF*(1.479-jet[trainInd[i]][2*j+1]+rJ)*hN/(2*1.479) )
                
                if (x_min < 1):
                    x_min = 1
                    trunc = 1
                if (y_min < 1):
                    y_min = 1
                    trunc = 1
                if (x_max > rF*hM):
                    x_max = rF*hM
                    trunc = 1
                if (y_max > rF*hN):
                    y_max = rF*hN
                    trunc = 1
                
                if (x_min>=x_max or y_min>=y_max):
                    print(i, jet[trainInd[i]][2*j+0], jet[trainInd[i]][2*j+1])
                    continue
                
#                 if ((x_max-x_min<40) or (y_max-y_min<50)):
#                     continue
                    
                nJet += 1
                                    
                # declaracion del objeto
                xmlF.write("".join("  <object>")+"\n")
                xmlF.write("".join("    <name>Jet</name>")+"\n")
                xmlF.write("".join("    <pose>Unspecified</pose>")+"\n")
                xmlF.write("".join("    <truncated>"+str(trunc)+"</truncated>")+"\n")
                xmlF.write("".join("    <difficult>0</difficult>")+"\n")
                xmlF.write("".join("    <bndbox>")+"\n")
                xmlF.write("".join("      <xmin>"+str(x_min)+"</xmin>")+"\n")
                xmlF.write("".join("      <ymin>"+str(y_min)+"</ymin>")+"\n")
                xmlF.write("".join("      <xmax>"+str(x_max)+"</xmax>")+"\n")
                xmlF.write("".join("      <ymax>"+str(y_max)+"</ymax>")+"\n")
                xmlF.write("".join("    </bndbox>")+"\n")
                xmlF.write("".join("  </object>")+"\n")
            # cierre
            xmlF.write("".join("</annotation>")+"\n")
            
            if (i>=1000):
                print(i+1, int(classes[trainInd[i]]), trainInd[i], nJet, jet[trainInd[i]])
        # cerrar la imagen
        xmlF.close()

1001 1 5166 6 [ 0.411 -0.721 -2.974  0.591 -2.949 -0.156 -2.336  0.047 -0.262 -0.828
 -1.389  0.843]
1002 0 1524 6 [-2.718  1.137  0.62   0.457 -0.603 -0.831 -2.395  0.151  2.672 -0.272
  3.029  0.436]
1003 1 5532 6 [ 0.047  0.957  3.119 -1.092 -2.614  0.352  2.998 -0.409 -2.473 -0.822
  0.629  0.859]
1004 1 3441 6 [ 1.247  0.605 -2.158  0.904 -1.837 -0.301 -1.186 -0.404  1.956 -0.363
  1.732  1.031]
1005 0 1062 6 [-2.799  1.103  0.4    0.951  2.907  0.819 -2.118  1.198  2.699  0.235
  0.023  0.138]
1006 1 5125 6 [-1.039  1.145  2.483  0.55   2.414 -0.623 -1.293 -0.798  1.849  0.561
  1.647 -0.36 ]
1007 0 1292 5 [ 0.234  0.705 -2.827 -0.177  0.561  0.26  -1.936  0.121  2.782 -0.385
  9.999  9.999]
1008 1 4183 6 [ 0.844  0.571 -2.542 -0.658 -2.268 -0.221 -1.728 -1.059 -2.381  0.673
 -1.142 -0.892]
1009 1 4152 6 [-0.637  1.16   2.552 -0.158  2.274  0.773 -1.236  1.037  1.459  0.634
 -0.575  0.359]
1010 0 899 3 [-0.42   0.218  2.686 -1.139  2.798 -0.416  9.999  9.999  9.999  9.999
  9.999